In [ ]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as k
print(k.__version__)

2.3.0


In [ ]:
!pip install soundfile

In [ ]:
def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import optimizers

In [ ]:
from keras.models import model_from_json
json_file= open('/content/drive/My Drive/speech_emotion/data/model.json','r')
loaded_model_json= json_file.read()
json_file.close()
Loaded_Model = model_from_json(loaded_model_json)
#Load weights into new model
Loaded_Model.load_weights("/content/drive/My Drive/speech_emotion/data/model.h5")
print("Loaded")

Loaded


In [ ]:
em=['neutral','happy','angry']

In [ ]:
neu = 0
hap = 0
ang = 0 
for file in glob.glob('/content/drive/My Drive/speech_emotion/neutral/*.wav'):
  features = np.array(extract_feature(file, mfcc=True, chroma=True, mel=True).reshape(1, -1))
    # predict
  f=np.expand_dims(features,axis=2)
  result = Loaded_Model.predict_classes(f)[0]
    # show the result !

  if (result == 0):
    neu = neu+1
  elif (result == 1):
    hap = hap+1
  else: ang = ang +1

  print("result: ",em[result])

print ('number neutral: ',neu)
print ('number happy: ',hap)
print ('number angry: ',ang)